In [1]:
import requests
import pandas as pd
pd.options.mode.chained_assignment = None
pd.set_option('display.float_format', lambda x: '%.1f' % x)

import base

with open('/Users/ilya/Desktop/keys/yahoo_finance.txt','r') as file:
    key = file.readlines()

KEY = key[0]

import datetime
from plotly import graph_objects as go

In [4]:
main_daily_df = base.get_stock_daily_price('ABBV',30)
my_tickers = ['EXC','BABA','JD','AMAT']

for ticker in my_tickers:
    mini_df = base.get_stock_daily_price(ticker,30)
    main_daily_df = main_daily_df.merge(mini_df, left_on = 'date', right_on = 'date')

fig = go.Figure()

fig.add_trace(go.Scatter(x=main_daily_df['date'], y=main_daily_df['abbv'], name='abbv_%', mode='lines+markers'))
fig.add_trace(go.Scatter(x=main_daily_df['date'], y=main_daily_df['exc'], name='exc_%', mode='lines+markers'))
fig.add_trace(go.Scatter(x=main_daily_df['date'], y=main_daily_df['baba'], name='baba_%', mode='lines+markers'))
fig.add_trace(go.Scatter(x=main_daily_df['date'], y=main_daily_df['jd'], name='jd_%', mode='lines+markers'))
fig.add_trace(go.Scatter(x=main_daily_df['date'], y=main_daily_df['amat'], name='amat_%', mode='lines+markers'))

fig.update_layout(title_text = 'Daily Stock Price Changes',
                  yaxis_title = 'Stock Price',
                  xaxis_title = 'Date',
                  height = 600, width = 1200)

fig.show()

In [ ]:
fig.write_html('plotly_page.html', full_html=False, include_plotlyjs='cdn')


In [2]:
main_weekly_df = base.get_stock_weekly_changes('ABBV',12)
my_tickers = ['EXC','BABA','JD','AMAT']

for ticker in my_tickers:
    mini_df = base.get_stock_weekly_changes(ticker,12)
    main_weekly_df = main_weekly_df.merge(mini_df, left_on = 'year-week', right_on = 'year-week')

In [3]:
fig = go.Figure()

fig.add_trace(go.Bar(x=main_weekly_df['year-week'], y=main_weekly_df['abbv'], name='abbv_%'))
fig.add_trace(go.Bar(x=main_weekly_df['year-week'], y=main_weekly_df['exc'], name='exc_%'))
fig.add_trace(go.Bar(x=main_weekly_df['year-week'], y=main_weekly_df['baba'], name='baba_%'))
fig.add_trace(go.Bar(x=main_weekly_df['year-week'], y=main_weekly_df['jd'], name='jd_%'))
fig.add_trace(go.Bar(x=main_weekly_df['year-week'], y=main_weekly_df['amat'], name='amat_%'))

# Change the bar mode
fig.update_layout(barmode='group', xaxis_title = 'Week-Year',height = 600, width = 1200)
fig.show()

In [2]:
def yearly_history_by_week(ticker_name):
    history_df = yf.download(tickers=ticker_name, period='1y').reset_index()[['Date','Close']]

    history_df.loc[:,'year'] = [date.isocalendar()[0] for date in history_df['Date']]
    history_df.loc[:,'week'] = [date.isocalendar()[1] for date in history_df['Date']]

    weekly_history_df = history_df[['year','week','Close']].groupby(['year','week']).mean().reset_index()

    weekly_history_df.loc[:,'close'] = weekly_history_df['Close'].round(1)
    weekly_history_df = weekly_history_df.drop(columns = ['Close']).rename(columns = {'close': ticker_name.lower()})
    
    return weekly_history_df

In [73]:
def visualize_quarterly_performance(ticker_name):
    bsx = yf.Ticker(ticker_name)
    quarter_balance_sheet = bsx.quarterly_balance_sheet
    kpi_df = pd.DataFrame(quarter_balance_sheet.loc['Total Current Assets'])
    kpi_df['Total Current Liabilities'] = quarter_balance_sheet.loc['Total Current Liabilities']
    kpi_df['Cash'] = quarter_balance_sheet.loc['Cash']

    kpi_df = kpi_df.reset_index().rename(columns = {'':'Quarter'})

    kpi_df.loc[:,'Assets_B'] = [round(revenue/1000000000,2) for revenue in kpi_df['Total Current Assets']]
    kpi_df.loc[:,'Liabilities_B'] = [round(earnings/1000000000,2) for earnings in kpi_df['Total Current Liabilities']]
    kpi_df.loc[:,'Cash_B'] = [round(earnings/1000000000,2) for earnings in kpi_df['Cash']]

    quarters = kpi_df['Quarter'].astype('str')

    fig = go.Figure(data=[
          go.Bar(name='Assets', x=quarters, y=kpi_df['Total Current Assets'], text = kpi_df['Assets_B']),
          go.Bar(name='Liabilities', x=quarters, y=kpi_df['Total Current Liabilities'], text = kpi_df['Liabilities_B']),
          go.Bar(name='Cash', x=quarters, y=kpi_df['Cash'], text = kpi_df['Cash_B'])
        ])

    fig.update_traces(textposition='outside', texttemplate='%{text}B')

    # Change the bar mode
    fig.update_layout(barmode='group', xaxis_title = 'Quarter',height = 500, width = 1000)
    fig.show()

In [74]:
visualize_quarterly_performance('BSX')

KeyboardInterrupt: 

In [64]:
def visualize_quarter_earnings(ticker_name):
    bsx = yf.Ticker(ticker_name)

    quarter_earnings = bsx.quarterly_earnings.reset_index()

    quarter_earnings.loc[:,'Revenue_B'] = [round(revenue/1000000000,2) for revenue in quarter_earnings['Revenue']]
    quarter_earnings.loc[:,'Earnings_B'] = [round(earnings/1000000000,2) for earnings in quarter_earnings['Earnings']]

    quarters = quarter_earnings['Quarter']

    fig = go.Figure(data=[
          go.Bar(name='Revenue', x=quarters, y=quarter_earnings['Revenue'], text = quarter_earnings['Revenue_B']),
          go.Bar(name='Earnings', x=quarters, y=quarter_earnings['Earnings'], text = quarter_earnings['Earnings_B'])
        ])

    fig.update_traces(textposition='outside', texttemplate='%{text}B')

    fig.update_layout(barmode='group', xaxis_title = 'Quarter',height = 500, width = 1000)

    return fig.show()

In [67]:
visualize_quarter_earnings('EXC')

In [6]:
my_portfolio_tickers = ['ABBV','BABA','JD','BSX']

history_df = yearly_history_by_week('EXC')

for company in my_portfolio_tickers:
    history_df = history_df.merge(yearly_history_by_week(company), left_on = ['year','week'], right_on = ['year','week'])
    print(company, end = ' ')
    time.sleep(3)

history_df.head(4)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
BSX 

,year,week,exc,abbv,baba,jd,bsx
0,2020,47,42.1,100.3,265.3,86.9,34.0
1,2020,48,42.1,103.8,276.1,88.8,33.6
2,2020,49,41.2,105.1,264.6,85.3,33.8
3,2020,50,41.2,107.2,264.7,82.3,34.0
